## Type of Cancer Classifiction

Dataset Source: https://www.kaggle.com/datasets/falgunipatel19/biomedical-text-publication-classification

##### Import Necessary Libraries

In [0]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, StructType, StructField, IntegerType, DoubleType
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

##### Versions of Libraries, Modules, Frameworks Used in This Project

In [0]:
print("Apache Spark version:", spark.version)
print("Spark NLP version:", sparknlp.version())

Apache Spark version: 3.2.1
Spark NLP version: 4.2.5


#### Create Functions Used Throughout This Project

##### Create Function to Ingest Dataset

In [0]:
def ingest_dataset(file_location: str, \
                   schema: StructType \
                  ) -> pyspark.sql.dataframe.DataFrame:
    '''
    This function returns a dataset aligned with the schema
    '''
    file_type = "csv"
    infer_schema = "false"
    first_row_is_header = "true"
    delimiter = ","
    
    df = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("sep", delimiter) \
      .schema(schema)\
      .load(file_location)
    
    return df

##### Various Data Preprocessing Functions Used in This Project

In [0]:
def remove_leading_chars(dataset: pyspark.sql.dataframe.DataFrame, \
                         column_name: str, \
                         char_to_remove: str = " "\
                        ) -> pyspark.sql.dataframe.DataFrame:
    '''
    This function removes leading characters (which are passed in as 'char_to_remove') from 
    '''
    dataset = dataset.withColumn(column_name, F.regexp_replace(column_name, f"^[{char_to_remove}]*", ''))
    return dataset
    
def remove_trailing_chars(dataset: pyspark.sql.dataframe.DataFrame, \
                         column_name: str, \
                         char_to_remove: str = " "\
                        ) -> pyspark.sql.dataframe.DataFrame:
    '''
    This function removes trailing characters (which are passed in as 'char_to_remove') from 
    '''
    dataset = dataset.withColumn(column_name, F.regexp_replace(column_name, f"[{char_to_remove}]*$", ""))
    return dataset

def add_text_length_column(dataset: pyspark.sql.dataframe.DataFrame, \
                           what_column: str, \
                           new_col_name: str\
                          ) -> pyspark.sql.dataframe.DataFrame:
    '''
    This function adds a column that calculates & displays the length of the
    column that is passed in ('what_column').
    '''
    dataset = dataset.withColumn(new_col_name, F.size(F.split(F.col(what_column), " ")))
    return dataset
     

def filter_labels(dataset: pyspark.sql.dataframe.DataFrame, \
                  label_vals_to_keep: [str], \
                  label_col_name: str = "label"\
                 ) -> pyspark.sql.dataframe.DataFrame:
    '''
    This function removes samples with unwanted values in the label column.
    '''
    filtered_df = dataset.filter(dataset[label_col_name].isin(label_vals_to_keep))
    return filtered_df

##### Create Function to Create Pipeline

In [0]:
def prepare_data(dataset: pyspark.sql.dataframe.DataFrame, \
                         all_features: [str], \
                         categorical_columns: [str], \
                         numerical_columns: [str], \
                         label = "label" \
                ) -> {}:
    '''
    This function creates the pipeline stages for classification analysis.
    '''
    
    # Define variables used within function
    stages = []
        
    # Prepare categorical features
    for cat in categorical_columns:
        stringIndexer = StringIndexer(inputCol = cat, outputCol = cat + "_index")
        encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()],\
                                outputCols=[cat + "_class_vec"])
        stages += [stringIndexer]
        stages += [encoder]
    
    # Use the vector assembler
    assembler_inputs = [c + "_class_vec" for c in categorical_columns] + numerical_columns
    assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")
    stages += [assembler]
    
    # using the pipeline created above, prepare the data for the model
    data_pipe = Pipeline().setStages(stages)
    data_fitted = data_pipe.fit(dataset)
    prepped_ds = data_fitted.transform(dataset)
    
    return prepped_ds

##### Create Metrics Evaluation Function

In [0]:
def evaluate_with_spark_metrics(dataset: pyspark.sql.dataframe.DataFrame, \
                                metrics: [str], \
                                model_name: str \
                               ) -> None:
    '''
    Calculate & display metrics for a multiclass classification analysis.
    '''
    print("+---------------------------------------------+")
    print("|  " + model_name.center(41) + "  |")
    print("+---------------------------------------------+")
    print("|   %s  |  %s   |" % ("Metric".rjust(20), "Value".ljust(14)))
    print("+---------------------------------------------+")
    for x in metrics:
        evaluator = MulticlassClassificationEvaluator(labelCol="label", \
                                                      predictionCol="prediction", \
                                                      metricName=x) 
        score = evaluator.evaluate(dataset)
        print("|   %s  |  %s   |" % (x.rjust(20), str(round(score, 6)).ljust(14)))
        print("+---------------------------------------------+")

#### Ingest & Preprocess Datasets

##### Ingest Dataset

In [0]:
data_file = "/FileStore/tables/alldata_1_for_kaggle.csv"

orig_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("label", StringType(), True),
    StructField("text", StringType(), True),
])

df = ingest_dataset(data_file, orig_schema)

df = df.drop("id")

labels_to_keep = ["Colon_Cancer", "Thyroid_Cancer", "Lung_Cancer"]

# Filter out non-compliant labels ('nulls')
df = filter_labels(df, labels_to_keep)

display(df)

label text Thyroid_Cancer Thyroid surgery in children in a single institution from Osama Ibrahim Almosallama Ali Aseerib Ahmed Alhumaida Ali S AlZahranic Saif Alsobhib Saud AlShanafeybFrom the aDepartment of Surgery College of Medicine Qassim University Buraidah Al Qassim Saudi Arabia bDepartment of Surgery King Faisal Specialist Hospital and Research Center Riyadh Saudi Arabia cDepartment of Medicine King Faisal Specialist Hospital and Research Center Riyadh Saudi Arabia Correspondence Dr Osama Ibrahim Almosallam Department of Surgery College of Medicine Qassim University PO Box Buraidah Al Qassim Saudi Arabia osama_iaahotmailcom ORCID orcid0000000290367564 Citation Almosallam OI Aseeri A Alhumaid A AlZahrani AS Alsobhi S AlShanafey S Thyroid surgery in children in a single institution from Ann Saudi Med Received January Accepted May Published August Copyright Copyright Annals of Saudi Medicine Saudi Arabia This is an access under the Creative Commons AttributionNonCommercialNoDerivatives International License CC BYNCND The details of which can be accessed at httpcreativecommons licensesbyncnd40Funding NoneBACKGROUND Data on thyroid surgery in children are scarceOBJECTIVE Analyze outcome data on thyroid surgery in a pediatric populationDESIGN Medical record reviewSETTING Tertiary health care institutionPATIENTS AND METHODS We collected demographic and clinical data on patients years or younger who had thyroid surgery in the period to Descriptive data are presentedMAIN OUTCOME MEASURES Indications for thyroidectomy thyroid pathology complications length of stay and radioactive iodine treatment and recurrencesSAMPLE SIZE RESULTS Of patients who underwent thyroidectomy procedures were females and the mean age at operation was years and were associated with multiple endocrine neoplasia type There was no history of radiation exposure Eightyone patients had fine needle aspiration FNA which correlated with the final histopathology in of cases Sixtysix patients had malignant cancer papillary of patients who had neck dissection had lymph node metastasis and had distant metastases to the lung Procedures included total thyroidectomy hemithyroidectomy completion and subtotal thyroidectomy Twentythree patients developed hypocalcemia permanent and had unilateral recurrent laryngeal nerve injury permanent Patients were followed up for a mean duration of months median months Of patients with thyroid cancer received radioactive iodine and had recurrence Malignancy is the commonest indication for thyroid surgery in children and FNA is highly diagnostic Hypocalcemia and recurrent laryngeal nerve injury are significant complications The recurrence rate in thyroid cancer is LIMITATIONS RetrospectiveCONFLICT OF INTEREST Noneoriginal ANN SAUDI MED JULYAUGUST WWWANNSAUDIMEDNET 0cThyroid diseases requiring surgery are relatively uncommon in children compared to adults The prevalence of palpable thyroid nodules in children ranges from to Sporadic welldifferentiated thyroid cancer is the most common endocrine malignancy in children accounting for of pediatric cancers in the prepubertal age group and up to of cancers in adolescents aged � year2 The most common indication for thyroid surgery in children varies among published studies but thyroidectomy for malignant conditions is rising38 Data in children throughout the world are relatively scarce The objective of this study was to analyze the clinical data and outcome of thyroid surgery in a large series of children treated at a single center at King Faisal Specialist Hospital and Research Center KFSHRC in RiyadhPATIENT AND METHODS With the approval of the Institutional Review Board IRB at KFSHRC the medical records of all patients years old and younger who underwent a thyroid surgery between and were retrospectively reviewed We elected to include patients up to the year to ensure a reasonable followup period Patients for the study were identified by a search of the operating room log for all procedures 

##### Data Preprocessing (Part 1)

In [0]:
# trim leading and trailing double quotes
df = remove_leading_chars(df, "text", "\"")
                      
df = remove_trailing_chars(df, "text", "\"")

# Filter out non-compliant labels ('nulls')
df = filter_labels(df, labels_to_keep)

# Add word length column for analysis
df = add_text_length_column(df, "text", "text_len")

# Convert Labels from string to integers
str_to_double_converter = {"Colon_Cancer" : "0", "Thyroid_Cancer" : "1", "Lung_Cancer" : "2"}

df = df.replace(str_to_double_converter, subset=["label"])

display(df)

label text text_len 1 Thyroid surgery in children in a single institution from Osama Ibrahim Almosallama Ali Aseerib Ahmed Alhumaida Ali S AlZahranic Saif Alsobhib Saud AlShanafeybFrom the aDepartment of Surgery College of Medicine Qassim University Buraidah Al Qassim Saudi Arabia bDepartment of Surgery King Faisal Specialist Hospital and Research Center Riyadh Saudi Arabia cDepartment of Medicine King Faisal Specialist Hospital and Research Center Riyadh Saudi Arabia Correspondence Dr Osama Ibrahim Almosallam Department of Surgery College of Medicine Qassim University PO Box Buraidah Al Qassim Saudi Arabia osama_iaahotmailcom ORCID orcid0000000290367564 Citation Almosallam OI Aseeri A Alhumaid A AlZahrani AS Alsobhi S AlShanafey S Thyroid surgery in children in a single institution from Ann Saudi Med Received January Accepted May Published August Copyright Copyright Annals of Saudi Medicine Saudi Arabia This is an access under the Creative Commons AttributionNonCommercialNoDerivatives International License CC BYNCND The details of which can be accessed at httpcreativecommons licensesbyncnd40Funding NoneBACKGROUND Data on thyroid surgery in children are scarceOBJECTIVE Analyze outcome data on thyroid surgery in a pediatric populationDESIGN Medical record reviewSETTING Tertiary health care institutionPATIENTS AND METHODS We collected demographic and clinical data on patients years or younger who had thyroid surgery in the period to Descriptive data are presentedMAIN OUTCOME MEASURES Indications for thyroidectomy thyroid pathology complications length of stay and radioactive iodine treatment and recurrencesSAMPLE SIZE RESULTS Of patients who underwent thyroidectomy procedures were females and the mean age at operation was years and were associated with multiple endocrine neoplasia type There was no history of radiation exposure Eightyone patients had fine needle aspiration FNA which correlated with the final histopathology in of cases Sixtysix patients had malignant cancer papillary of patients who had neck dissection had lymph node metastasis and had distant metastases to the lung Procedures included total thyroidectomy hemithyroidectomy completion and subtotal thyroidectomy Twentythree patients developed hypocalcemia permanent and had unilateral recurrent laryngeal nerve injury permanent Patients were followed up for a mean duration of months median months Of patients with thyroid cancer received radioactive iodine and had recurrence Malignancy is the commonest indication for thyroid surgery in children and FNA is highly diagnostic Hypocalcemia and recurrent laryngeal nerve injury are significant complications The recurrence rate in thyroid cancer is LIMITATIONS RetrospectiveCONFLICT OF INTEREST Noneoriginal ANN SAUDI MED JULYAUGUST WWWANNSAUDIMEDNET 0cThyroid diseases requiring surgery are relatively uncommon in children compared to adults The prevalence of palpable thyroid nodules in children ranges from to Sporadic welldifferentiated thyroid cancer is the most common endocrine malignancy in children accounting for of pediatric cancers in the prepubertal age group and up to of cancers in adolescents aged � year2 The most common indication for thyroid surgery in children varies among published studies but thyroidectomy for malignant conditions is rising38 Data in children throughout the world are relatively scarce The objective of this study was to analyze the clinical data and outcome of thyroid surgery in a large series of children treated at a single center at King Faisal Specialist Hospital and Research Center KFSHRC in RiyadhPATIENT AND METHODS With the approval of the Institutional Review Board IRB at KFSHRC the medical records of all patients years old and younger who underwent a thyroid surgery between and were retrospectively reviewed We elected to include patients up to the year to ensure a reasonable followup period Patients for the study were identified by a search of the operating room log for all procedures invo

##### Data Preprocessing (Part 2)

In [0]:
df = df.withColumn("label", F.col("label").cast(DoubleType()))
df.printSchema()

root
 |-- label: double (nullable = true)
 |-- text: string (nullable = true)
 |-- text_len: integer (nullable = false)



##### Split Dataset into Training & Testing Datasets

In [0]:
train_ds, test_ds = df.randomSplit(weights=[0.80, 0.20], seed=42)

train_ds = train_ds.persist()
test_ds = test_ds.persist()

print(f"There are {train_ds.count()} samples in the training dataset.")
print(f"There are {test_ds.count()} samples in the testing dataset.")

There are 6075 samples in the training dataset.
There are 1495 samples in the testing dataset.


#### Create & Train Model

##### Build Pipeline Stages

In [0]:
# Document Assembler
doc = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

# USE Embeddings
embeddings = UniversalSentenceEncoder.pretrained() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")

# Deep Learning Approach Classifier
clf = ClassifierDLApproach() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("class") \
    .setLabelColumn("label") \
    .setMaxEpochs(12)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


##### Build Pipeline

In [0]:
cancer_clf_pipe = Pipeline().setStages([
    doc, 
    embeddings, 
    clf
])

##### Train Model Using Training Dataset

In [0]:
cancer_clf_model = cancer_clf_pipe.fit(train_ds)

#### Evaluate Model

##### Generate Predictions Using Testing Dataset

In [0]:
predictions = cancer_clf_model.transform(test_ds)

##### Prepare Predictions for Evaluation Function

In [0]:
preds = predictions.select(F.col("label").alias("label"), F.col("class.result").alias("prediction"))
preds = preds.withColumn("prediction", F.col("prediction").getItem(0))
preds = preds.withColumn("prediction", F.col("prediction").cast(DoubleType()))

##### Calculate & Display Metrics Results

In [0]:
metrics_to_eval = ["accuracy", "f1", "weightedPrecision", "weightedRecall"]
                   
evaluate_with_spark_metrics(preds, \
                            metrics_to_eval, \
                            "Metrics for Cancer Classification")

+---------------------------------------------+
|      Metrics for Cancer Classification      |
+---------------------------------------------+
|                 Metric  |  Value            |
+---------------------------------------------+
|               accuracy  |  0.862876         |
+---------------------------------------------+
|                     f1  |  0.86247          |
+---------------------------------------------+
|      weightedPrecision  |  0.862128         |
+---------------------------------------------+
|         weightedRecall  |  0.862876         |
+---------------------------------------------+


##### End Spark Session

In [0]:
train_ds = train_ds.unpersist()
test_ds = test_ds.unpersist()

spark.stop()

### Notes & Other Takeaways From This Project
****
- The results are pretty good. The weighted precision and weighted recall values are very close, which is a good sign as well!
****